In [2]:
import os
import getpass
from uuid import uuid4

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY")
os.environ["COHERE_API_KEY"] = getpass.getpass("COHERE_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")
os.environ["LANGCHAIN_PROJECT"] = f"AIM - CERT_CHALLENGE - {uuid4().hex[0:8]}"

In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bsmith53\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\bsmith53\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
import nest_asyncio

nest_asyncio.apply()

In [5]:
!mkdir data_new

A subdirectory or file data_new already exists.


In [6]:
!curl https://www.tidyverse.org/help/ -o data_new/tidy_help.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 18445  100 18445    0     0  53842      0 --:--:-- --:--:-- --:--:-- 54250


In [7]:
!curl https://www.tidyverse.org/blog/2025/04/learn-tidyverse-ai/ -o data_new/learn_tidyverse_ai.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 75782  100 75782    0     0   340k      0 --:--:-- --:--:-- --:--:--  342k


In [8]:
!curl https://www.tidyverse.org/blog/2025/02/tidymodels-2025-q1/ -o data_new/tidymodels_2025_q1.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 38195  100 38195    0     0   165k      0 --:--:-- --:--:-- --:--:--  167k


In [9]:
!curl https://www.tidyverse.org/blog/2025/01/experiments-llm/ -o data_new/experiments_llm.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 35796  100 35796    0     0   174k      0 --:--:-- --:--:-- --:--:--  176k


In [10]:
!curl https://www.tidyverse.org/packages/ -o data_new/packages.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 26673  100 26673    0     0   140k      0 --:--:-- --:--:-- --:--:--  143k


In [11]:
from langchain_community.document_loaders import DirectoryLoader

path = "data_new/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


In [12]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.arxiv.tool import ArxivQueryRun


tavily_tool = TavilySearchResults(max_results=5)

In [14]:
path = "data_new/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=750, chunk_overlap=100, length_function = len)
split_documents = text_splitter.split_documents(docs)
len(split_documents)

113

In [16]:
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="Snowflake/snowflake-arctic-embed-m")


In [18]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="tidy_docs",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="tidy_docs",
    embedding=embeddings,
)

In [19]:
_ = vector_store.add_documents(documents=split_documents)

In [20]:

retriever = vector_store.as_retriever(search_kwargs={"k": 5})

In [21]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank


def retrieve(state):
  retrieved_docs = retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

In [22]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
You are a helpful assistant who answers questions based on provided context. You must only use the provided context, and cannot use your own knowledge.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [23]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

In [24]:
def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
  response = llm.invoke(messages)
  return {"response" : response.content}

In [25]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
  question: str
  context: List[Document]
  response: str

In [26]:
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [27]:
response = graph.invoke({"question" : "How are LLM agents useful in R coding?"})

In [28]:
response["response"]

"LLM agents are useful in R coding primarily because they have direct access to your R environment. This means they can see the data frames you have loaded, their columns, and the column types, which reduces friction. Unlike language-agnostic tools where you have to manually print and share variable information, these LLM packages can automatically use your environment's context, making assistance with coding tasks smoother and more effective.\n\nAdditionally, these LLM tools can write code directly into your R files, eliminating the need for copy-pasting model responses. This streamlines the workflow when the suggested code is accurate. However, you still need to check the generated code for errors manually, as some suggested arguments might be incorrect or ignored by R without clear errors.\n\nIn summary, LLM agents enhance productivity in R coding by:\n- Utilizing direct access to the R environment for context-aware assistance.\n- Automatically incorporating changes into your files,

In [29]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_core.runnables import Runnable



In [30]:
tool_belt = [
    tavily_tool,
]

In [31]:
from langchain_openai import ChatOpenAI

cert_model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0
)

In [32]:
cert_model = cert_model.bind_tools(tool_belt)

In [33]:
from typing import Annotated, List, Tuple, Union
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
import operator
from langchain_core.messages import BaseMessage
from langchain_core.documents import Document

class AgentState(TypedDict):
    messages: Annotated[list, add_messages]
    context: List[Document]

In [34]:
from langgraph.prebuilt import ToolNode

def call_model(state):
  messages = state["messages"]
  response = cert_model.invoke(messages)
  return {"messages" : [response],
          "context" : state.get("context", [])
    }

tool_node = ToolNode(tool_belt)

print(tool_node)

tools(tags=None, recurse=True, explode_args=False, func_accepts_config=True, func_accepts={'store': ('__pregel_store', None)}, tools_by_name={'tavily_search_results_json': TavilySearchResults(api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********')))}, tool_to_state_args={'tavily_search_results_json': {}}, tool_to_store_arg={'tavily_search_results_json': None}, handle_tool_errors=True, messages_key='messages')


In [35]:
from langgraph.graph import StateGraph, END

uncompiled_graph = StateGraph(AgentState)

uncompiled_graph.add_node("agent", call_model)
uncompiled_graph.add_node("action", tool_node)

In [36]:
uncompiled_graph.set_entry_point("agent")

In [37]:
def should_continue(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  return END

uncompiled_graph.add_conditional_edges(
    "agent",
    should_continue
)

In [38]:
uncompiled_graph.add_edge("action", "agent")

In [39]:
compiled_graph = uncompiled_graph.compile()

In [40]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="What is the current captian of the WInnipeg Jets?")]}

async for chunk in compiled_graph.astream(inputs, stream_mode = "updates"):
    for node, values in chunk.items():
        print(f"Receiiving update from nods: '{node}")
        print(values["messages"])
        print("\n\n")

Receiiving update from nods: 'agent
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rbFfYnujb1qK2gyhIm2rolDw', 'function': {'arguments': '{"query":"current captain of the Winnipeg Jets"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 92, 'total_tokens': 116, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_79b79be41f', 'id': 'chatcmpl-BWmCSfVfPYJIIOpZ1K4e2hEgEE5ar', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--f88ee6ff-91f3-4d64-a1bd-ea61dae8e97a-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current captain of the Winnipeg Jets'}, 'id': 'call_rbFfYnujb1qK2gyhIm2rolDw'

In [ ]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="What is the current state of AI agents?")]}

async for chunk in compiled_graph.astream(inputs, stream_mode = "updates"):
    for node, values in chunk.items():
        print(f"Receiiving update from nodes: '{node}")
        if node == 'action':
            print(f"Tool used: {values['messages'][0].name}")
        print(values["messages"])
        
        print("\n\n")

In [41]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/5 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/5 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/5 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/19 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/41 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [42]:

dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is CRAN and why is it important for R pac...,[Tidyverse Packages Blog Learn Help Contribute...,"CRAN is the Comprehensive R Archive Network, a...",single_hop_specifc_query_synthesizer
1,How did the Posit LLM contribute to the develo...,"[Background Twice a year, the tidyverse team s...",The Posit LLM contributed to the development o...,single_hop_specifc_query_synthesizer
2,Can you explain the significance of the tidyve...,"[gander—that came out of that exploration, and...",The tidyverse is significant in R package deve...,single_hop_specifc_query_synthesizer
3,What are the advantages of using Positron IDE ...,[rstudioapi has its ups and downs. As for the ...,The advantages of using Positron IDE include i...,single_hop_specifc_query_synthesizer
4,How can data cleaning be effectively integrate...,[<1-hop>\n\nQuantile regression in parsnip One...,To effectively integrate data cleaning with da...,multi_hop_abstract_query_synthesizer
5,How can generative AI tools assist in data vis...,[<1-hop>\n\nCase study 1: Reshaping and plotti...,Generative AI tools can assist in data visuali...,multi_hop_abstract_query_synthesizer
6,What is the relationship between the Tidyverse...,[<1-hop>\n\nTidyverse Packages Blog Learn Help...,The Tidyverse is a collection of R packages th...,multi_hop_abstract_query_synthesizer
7,How can the Tidyverse enhance data visualizati...,[<1-hop>\n\nTidyverse Packages Blog Learn Help...,The Tidyverse enhances data visualization thro...,multi_hop_abstract_query_synthesizer
8,How did clipal and ellmer contribute to improv...,[<1-hop>\n\ngander—that came out of that explo...,Clipal and ellmer significantly improved produ...,multi_hop_specific_query_synthesizer
9,How can I use tidyr to reshape the climate dat...,[<1-hop>\n\nCase study 3: Web scraping For the...,To reshape the climate data scraped from the N...,multi_hop_specific_query_synthesizer


In [43]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

base_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | llm | StrOutputParser(), "context": itemgetter("context")}
)

In [44]:
baseline_data = dataset

for test_row in baseline_data:
  response = base_rag_chain.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [45]:
baseline_data.to_pandas()

,user_input,retrieved_contexts,reference_contexts,response,reference,synthesizer_name
0,What is CRAN and why is it important for R pac...,"[Get help\n\nIf you’re asking for R help, repo...",[Tidyverse Packages Blog Learn Help Contribute...,The provided context does not contain specific...,"CRAN is the Comprehensive R Archive Network, a...",single_hop_specifc_query_synthesizer
1,How did the Posit LLM contribute to the develo...,[clipal was a huge boost for us in the most re...,"[Background Twice a year, the tidyverse team s...",The Posit LLM contributed to the development o...,The Posit LLM contributed to the development o...,single_hop_specifc_query_synthesizer
2,Can you explain the significance of the tidyve...,[Contents\n\nThe tidyverse is proudly supporte...,"[gander—that came out of that exploration, and...",The tidyverse is significant in the context of...,The tidyverse is significant in R package deve...,single_hop_specifc_query_synthesizer
3,What are the advantages of using Positron IDE ...,"[What’s next?\n\nFor now, all of these package...",[rstudioapi has its ups and downs. As for the ...,The advantages of using Positron IDE for R dev...,The advantages of using Positron IDE include i...,single_hop_specifc_query_synthesizer
4,How can data cleaning be effectively integrate...,[Contents\n\nThe tidyverse is proudly supporte...,[<1-hop>\n\nQuantile regression in parsnip One...,Data cleaning can be effectively integrated wi...,To effectively integrate data cleaning with da...,multi_hop_abstract_query_synthesizer
5,How can generative AI tools assist in data vis...,[ggplot2\n\nggplot2 is a system for declarativ...,[<1-hop>\n\nCase study 1: Reshaping and plotti...,Generative AI tools can assist in data visuali...,Generative AI tools can assist in data visuali...,multi_hop_abstract_query_synthesizer
6,What is the relationship between the Tidyverse...,[Organizing Tidy Dev Day issues.\n\nOrganizing...,[<1-hop>\n\nTidyverse Packages Blog Learn Help...,The relationship between the Tidyverse and qua...,The Tidyverse is a collection of R packages th...,multi_hop_abstract_query_synthesizer
7,How can the Tidyverse enhance data visualizati...,"[Get help\n\nIf you’re asking for R help, repo...",[<1-hop>\n\nTidyverse Packages Blog Learn Help...,The Tidyverse enhances data visualization by p...,The Tidyverse enhances data visualization thro...,multi_hop_abstract_query_synthesizer
8,How did clipal and ellmer contribute to improv...,[Contents\n\nThe tidyverse is proudly supporte...,[<1-hop>\n\ngander—that came out of that explo...,Clipal and Ellmer contributed to improving pro...,Clipal and ellmer significantly improved produ...,multi_hop_specific_query_synthesizer
9,How can I use tidyr to reshape the climate dat...,[We only have the first six months (the first ...,[<1-hop>\n\nCase study 3: Web scraping For the...,To use **tidyr** to reshape the climate data s...,To reshape the climate data scraped from the N...,multi_hop_specific_query_synthesizer


In [46]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(baseline_data.to_pandas())

In [47]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))

In [49]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

{'context_recall': 0.5258, 'faithfulness': 0.8521, 'factual_correctness(mode=f1)': 0.5667, 'answer_relevancy': 0.8159, 'context_entity_recall': 0.3222, 'noise_sensitivity(mode=relevant)': 0.2472}

In [50]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import BSHTMLLoader

path = "data_new/"
text_loader = DirectoryLoader(path, glob="*.html", loader_cls=BSHTMLLoader)

In [51]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 750,
    chunk_overlap  = 100,
    length_function = len
)

In [52]:
training_documents = text_splitter.split_documents(text_loader.load())

In [53]:
len(training_documents)

122

In [54]:
import uuid

id_set = set()

for document in training_documents:
  id = str(uuid.uuid4())
  while id in id_set:
    id = uuid.uuid4()
  id_set.add(id)
  document.metadata["id"] = id

In [55]:
training_split_documents = training_documents[:len(training_documents) - 24]
val_split_documents = training_documents[len(training_documents) - 24:122-12]
test_split_documents = training_documents[122-12:]

In [56]:
from langchain_openai import ChatOpenAI

qa_chat_model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

In [57]:
from langchain_core.prompts import ChatPromptTemplate

qa_prompt = """\
Given the following context, you must generate questions based on only the provided context.

You are to generate {n_questions} questions which should be provided in the following format:

1. QUESTION #1
2. QUESTION #2
...

Context:
{context}
"""

qa_prompt_template = ChatPromptTemplate.from_template(qa_prompt)

In [58]:
question_generation_chain = qa_prompt_template | qa_chat_model

In [60]:
import tqdm
import asyncio

"""
Sample Usage of TQDM:

for i in tqdm.tqdm(range(10)):
  time.sleep(1)
"""

async def create_questions(documents, n_questions):

    questions = {}
    relevant_docs = {}

    for doc in tqdm.tqdm(documents, desc="Generating questions"):
      # Oreoare tge uoyt for the chain
      input_context = doc.page_content
      doc_id = doc.metadata["id"]
      
      # Call the question generating chain
      response = await question_generation_chain.ainvoke({"context": input_context, "n_questions": n_questions})
      
      # Extract questions
      generated_questions = response.content.split("\n")
      generated_questions = [q.strip() for q in generated_questions if q.strip()]
      
      # Some inputs might be number like "1. What is ...?", so clean numbering
      cleaned_questions = []
      for q in generated_questions:
        if q[0].isdigit() and q[1] == '.':
          cleaned_questions.append(q[2:].strip())
        elif q[0].isdigit() and q[1] == ' ':
          cleaned_questions.append(q[1:].strip())
        else:
          cleaned_questions.append(q)
      
      #Now save each question
      for q in cleaned_questions:
        question_id = str(uuid.uuid4())
        questions[question_id] = q
        relevant_docs[question_id] = [doc_id]

    return questions, relevant_docs

In [61]:
training_questions, training_relevant_contexts = await create_questions(training_split_documents, 2)

Generating questions: 100%|██████████| 98/98 [01:39<00:00,  1.01s/it]


We'll use the function to generate training, validation, and test data.

In [62]:
val_questions, val_relevant_contexts = await create_questions(val_split_documents, 2)

Generating questions: 100%|██████████| 12/12 [00:11<00:00,  1.02it/s]


In [63]:
test_questions, test_relevant_contexts = await create_questions(test_split_documents, 2)

Generating questions: 100%|██████████| 12/12 [00:10<00:00,  1.11it/s]


In [64]:
import json

training_corpus = {train_item.metadata["id"] : train_item.page_content for train_item in training_split_documents}

train_dataset = {
    "questions" : training_questions,
    "relevant_contexts" : training_relevant_contexts,
    "corpus" : training_corpus
}

with open("training_dataset.jsonl", "w") as f:
  json.dump(train_dataset, f)

In [65]:
val_corpus = {val_item.metadata["id"] : val_item.page_content for val_item in val_split_documents}

val_dataset = {
    "questions" : val_questions,
    "relevant_contexts" : val_relevant_contexts,
    "corpus" : val_corpus
}

with open("val_dataset.jsonl", "w") as f:
  json.dump(val_dataset, f)

In [66]:
train_corpus = {test_item.metadata["id"] : test_item.page_content for test_item in test_split_documents}

test_dataset = {
    "questions" : test_questions,
    "relevant_contexts" : test_relevant_contexts,
    "corpus" : train_corpus
}

with open("test_dataset.jsonl", "w") as f:
  json.dump(test_dataset, f)

In [ ]:
#!pip install -qU sentence_transformers datasets pyarrow

In [67]:
from sentence_transformers import SentenceTransformer

model_id = "Snowflake/snowflake-arctic-embed-m"
model = SentenceTransformer(model_id)

We'll grab some necessary imports from `sentence_transformers` and `torch`.

> NOTE: PyTorch (`torch`) is a popular machine learning library - while we don't go very deep into PyTorch it's an incredibly powerful and interesting library! Please read more about it [here](https://pytorch.org/tutorials/beginner/basics/intro.html)!

In [68]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sentence_transformers import InputExample

In [69]:
BATCH_SIZE = 10

In [70]:
corpus = train_dataset['corpus']
queries = train_dataset['questions']
relevant_docs = train_dataset['relevant_contexts']

examples = []
for query_id, query in queries.items():
    doc_id = relevant_docs[query_id][0]
    text = corpus[doc_id]
    example = InputExample(texts=[query, text])
    examples.append(example)

In [71]:
loader = DataLoader(
    examples, batch_size=BATCH_SIZE
)

In [72]:
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss

matryoshka_dimensions = [768, 512, 256, 128, 64]
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

Now we can set-up our evaluator.

> NOTE: Due to the formatting of our dataset - this is all we have to do!

In [73]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

corpus = val_dataset['corpus']
queries = val_dataset['questions']
relevant_docs = val_dataset['relevant_contexts']

evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)

In [74]:
EPOCHS = 10

In [75]:
import wandb
wandb.init(mode="disabled")

In [76]:
warmup_steps = int(len(loader) * EPOCHS * 0.1)

model.fit(
    train_objectives=[(loader, train_loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path='finetuned_arctic_ft_cert',
    show_progress_bar=True,
    evaluator=evaluator,
    evaluation_steps=50
)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
c:\Users\bsmith53\AIE6_NEW_TEST\AIE6\Certification_Challenge\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss,Cosine Accuracy@1,Cosine Accuracy@3,Cosine Accuracy@5,Cosine Accuracy@10,Cosine Precision@1,Cosine Precision@3,Cosine Precision@5,Cosine Precision@10,Cosine Recall@1,Cosine Recall@3,Cosine Recall@5,Cosine Recall@10,Cosine Ndcg@10,Cosine Mrr@10,Cosine Map@100
20,No log,No log,0.750000,0.958333,0.958333,1.000000,0.750000,0.319444,0.191667,0.100000,0.750000,0.958333,0.958333,1.000000,0.889877,0.853175,0.853175
40,No log,No log,0.833333,0.916667,1.000000,1.000000,0.833333,0.305556,0.200000,0.100000,0.833333,0.916667,1.000000,1.000000,0.919975,0.893750,0.893750
50,No log,No log,0.833333,0.916667,0.958333,1.000000,0.833333,0.305556,0.191667,0.100000,0.833333,0.916667,0.958333,1.000000,0.918698,0.892361,0.892361
60,No log,No log,0.833333,0.916667,1.000000,1.000000,0.833333,0.305556,0.200000,0.100000,0.833333,0.916667,1.000000,1.000000,0.919975,0.893750,0.893750
80,No log,No log,0.833333,0.916667,1.000000,1.000000,0.833333,0.305556,0.200000,0.100000,0.833333,0.916667,1.000000,1.000000,0.919975,0.893750,0.893750
100,No log,No log,0.833333,0.958333,1.000000,1.000000,0.833333,0.319444,0.200000,0.100000,0.833333,0.958333,1.000000,1.000000,0.922863,0.897222,0.897222
120,No log,No log,0.833333,0.916667,1.000000,1.000000,0.833333,0.305556,0.200000,0.100000,0.833333,0.916667,1.000000,1.000000,0.919975,0.893750,0.893750
140,No log,No log,0.833333,0.916667,0.958333,1.000000,0.833333,0.305556,0.191667,0.100000,0.833333,0.916667,0.958333,1.000000,0.916872,0.890278,0.890278
150,No log,No log,0.833333,0.916667,0.958333,1.000000,0.833333,0.305556,0.191667,0.100000,0.833333,0.916667,0.958333,1.000000,0.916872,0.890278,0.890278
160,No log,No log,0.833333,0.916667,0.958333,1.000000,0.833333,0.305556,0.191667,0.100000,0.833333,0.916667,0.958333,1.000000,0.916872,0.890278,0.890278


In [77]:
from huggingface_hub import notebook_login

notebook_login()

In [78]:
hf_username = "bsmith3715"

In [79]:
import uuid

model.push_to_hub(f"{hf_username}/legal-ft-cert-challenge_final")

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

'https://huggingface.co/bsmith3715/legal-ft-cert-challenge_final/commit/e99afaf53a5a196e682fbcb13bc159a010141a25'

In [80]:
import pandas as pd

from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.documents import Document

In [81]:
path = "data_new/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


In [82]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=750, chunk_overlap=100)
split_documents = text_splitter.split_documents(docs)
len(split_documents)

113

In [83]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="finetuned_arctic_ft_cert")

Some weights of BertModel were not initialized from the model checkpoint at finetuned_arctic_ft_cert and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [84]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="tidy_docs",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)

vector_store_ft = QdrantVectorStore(
    client=client,
    collection_name="tidy_docs",
    embedding=embeddings,
)

In [85]:
_ = vector_store_ft.add_documents(documents=split_documents)

In [86]:

retriever_finetune = vector_store_ft.as_retriever(search_kwargs={"k": 5})

In [87]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

def retrieve_adjusted(state):
  compressor = CohereRerank(model="rerank-v3.5", top_n=10)
  compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever_finetune, search_kwargs={"k": 5}
  )
  retrieved_docs = compression_retriever.invoke(state["question"])
  return {"context" : retrieved_docs}



#
  #retrieved_docs = retriever.invoke(state["question"])
  #return {"context" : retrieved_docs}

In [88]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
You are a helpful assistant who answers questions based on provided context. You must only use the provided context, and cannot use your own knowledge.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [89]:
from langchain_openai import ChatOpenAI

ft_llm = ChatOpenAI(model="gpt-4o-mini")

In [90]:
def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
  response = ft_llm.invoke(messages)
  return {"response" : response.content}

In [91]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
  question: str
  context: List[Document]
  response: str

In [92]:
graph_builder = StateGraph(State).add_sequence([retrieve_adjusted, generate])
graph_builder.add_edge(START, "retrieve_adjusted")
graph = graph_builder.compile()

In [94]:
response = graph.invoke({"question" : "How are LLM agents useful in R coding?"})

In [95]:
response["response"]

"LLM agents are useful in R coding because they can directly interact with your R environment, allowing them to access loaded data frames and understand their columns and types. This capability reduces the friction typically associated with other LLM code-assist tools, as users do not need to manually print and paste variable information for the model to work effectively.\n\nAdditionally, R packages like `ellmer` enable users to initialize a chat with models like OpenAI's GPT or locally hosted models, making it easier for R users to utilize large language models directly in their coding workflow. This direct access means that users can quickly get assistance with code corrections or updates, as demonstrated by the `clipal1` package, which allows users to select erroring code, invoke the LLM, and inline the updated code.\n\nHowever, it's worth noting that LLM packages in R write directly to files, which can be both a benefit—eliminating the need for copy-pasting—and a drawback, as it ma

In [96]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_core.runnables import Runnable


In [97]:
from langchain_openai import ChatOpenAI

cert_model = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

In [98]:
tool_belt = [
    tavily_tool,
]

In [99]:
cert_model = cert_model.bind_tools(tool_belt)

In [100]:
from typing import Annotated, List, Tuple, Union
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
import operator
from langchain_core.messages import BaseMessage
from langchain_core.documents import Document

class AgentState(TypedDict):
    messages: Annotated[list, add_messages]
    context: List[Document]

In [101]:
from langgraph.prebuilt import ToolNode

def call_model(state):
  messages = state["messages"]
  response = cert_model.invoke(messages)
  return {"messages" : [response],
          "context" : state.get("context", [])
    }

tool_node = ToolNode(tool_belt)

print(tool_node)

tools(tags=None, recurse=True, explode_args=False, func_accepts_config=True, func_accepts={'store': ('__pregel_store', None)}, tools_by_name={'tavily_search_results_json': TavilySearchResults(api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********')))}, tool_to_state_args={'tavily_search_results_json': {}}, tool_to_store_arg={'tavily_search_results_json': None}, handle_tool_errors=True, messages_key='messages')


In [102]:
from langgraph.graph import StateGraph, END

uncompiled_graph = StateGraph(AgentState)

uncompiled_graph.add_node("agent", call_model)
uncompiled_graph.add_node("action", tool_node)

In [103]:
uncompiled_graph.set_entry_point("agent")

In [104]:
def should_continue(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  return END

uncompiled_graph.add_conditional_edges(
    "agent",
    should_continue
)

In [105]:
uncompiled_graph.add_edge("action", "agent")

In [106]:
compiled_graph_w_fine_tune = uncompiled_graph.compile()
print(compiled_graph_w_fine_tune)

In [107]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

finetune_rag_chain = (
    {"context": itemgetter("question") | retriever_finetune, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | ft_llm | StrOutputParser(), "context": itemgetter("context")}
)

In [108]:
finetune_data = dataset

for test_row in finetune_data:
  response = finetune_rag_chain.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [109]:
finetune_data.to_pandas()

,user_input,retrieved_contexts,reference_contexts,response,reference,synthesizer_name
0,What is CRAN and why is it important for R pac...,[The reprex package\n\nWhen creating a reprex ...,[Tidyverse Packages Blog Learn Help Contribute...,"CRAN, which stands for the Comprehensive R Arc...","CRAN is the Comprehensive R Archive Network, a...",single_hop_specifc_query_synthesizer
1,How did the Posit LLM contribute to the develo...,[The week before our most recent spring cleani...,"[Background Twice a year, the tidyverse team s...",The Posit LLM contributed to the development o...,The Posit LLM contributed to the development o...,single_hop_specifc_query_synthesizer
2,Can you explain the significance of the tidyve...,[Tidyverse\n\nPackages Blog Learn Help Contrib...,"[gander—that came out of that exploration, and...",The tidyverse plays a significant role in R pa...,The tidyverse is significant in R package deve...,single_hop_specifc_query_synthesizer
3,What are the advantages of using Positron IDE ...,"[When the response is returned, replace the co...",[rstudioapi has its ups and downs. As for the ...,The advantages of using Positron IDE for R dev...,The advantages of using Positron IDE include i...,single_hop_specifc_query_synthesizer
4,How can data cleaning be effectively integrate...,[Would you like me to explain any part of this...,[<1-hop>\n\nQuantile regression in parsnip One...,To effectively integrate data cleaning with da...,To effectively integrate data cleaning with da...,multi_hop_abstract_query_synthesizer
5,How can generative AI tools assist in data vis...,[Tidyverse\n\nPackages Blog Learn Help Contrib...,[<1-hop>\n\nCase study 1: Reshaping and plotti...,Generative AI tools can assist in data visuali...,Generative AI tools can assist in data visuali...,multi_hop_abstract_query_synthesizer
6,What is the relationship between the Tidyverse...,[Organizing Tidy Dev Day issues.\n\nOrganizing...,[<1-hop>\n\nTidyverse Packages Blog Learn Help...,The relationship between the Tidyverse and qua...,The Tidyverse is a collection of R packages th...,multi_hop_abstract_query_synthesizer
7,How can the Tidyverse enhance data visualizati...,[Tidyverse\n\nPackages Blog Learn Help Contrib...,[<1-hop>\n\nTidyverse Packages Blog Learn Help...,The Tidyverse enhances data visualization prim...,The Tidyverse enhances data visualization thro...,multi_hop_abstract_query_synthesizer
8,How did clipal and ellmer contribute to improv...,"[clipal itself is now superseded by pal, a mor...",[<1-hop>\n\ngander—that came out of that explo...,Clipal and Ellmer significantly contributed to...,Clipal and ellmer significantly improved produ...,multi_hop_specific_query_synthesizer
9,How can I use tidyr to reshape the climate dat...,[# Convert the table to a data frame\nclimate_...,[<1-hop>\n\nCase study 3: Web scraping For the...,To reshape the climate data you've scraped fro...,To reshape the climate data scraped from the N...,multi_hop_specific_query_synthesizer


In [110]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(finetune_data.to_pandas())

In [111]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))

In [112]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

{'context_recall': 0.5889, 'faithfulness': 0.7677, 'factual_correctness(mode=f1)': 0.5775, 'answer_relevancy': 0.9735, 'context_entity_recall': 0.4178, 'noise_sensitivity(mode=relevant)': 0.2349}

FIRST_RUN  - {'context_recall': 0.5258, 'faithfulness': 0.8521, 'factual_correctness(mode=f1)': 0.5667, 'answer_relevancy': 0.8159, 'context_entity_recall': 0.3222, 'noise_sensitivity(mode=relevant)': 0.2472}

FINE_TUNED - {'context_recall': 0.5889, 'faithfulness': 0.7677, 'factual_correctness(mode=f1)': 0.5775, 'answer_relevancy': 0.9735, 'context_entity_recall': 0.4178, 'noise_sensitivity(mode=relevant)': 0.2349